# Love Behavior with Human Following

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='60%')
display_depth = widgets.Image(format='jpeg', width='60%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO model for human detection**

In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

**Step 3: Initialize camera and implement love behavior with human following**

The robot will:
- Follow detected human by adjusting orientation (turn left/right)
- Approach to maintain optimal distance (450-600mm)
- Stop when human is at comfortable distance
- Only follow humans, not all objects

In [ ]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

**Step 4: Implement love behavior with human following callback**

In [ ]:
def love_callback(change):
    frame = change['new']
    
    # Run YOLO inference to detect humans
    results = model(frame, verbose=False)
    
    # Prepare depth colormap for display
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    conf_threshold = 0.5
    human_detected = False
    closest_distance = float('inf')
    closest_bbox = None
    
    # Check for human detections (class 0)
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human subject
                if result.boxes.conf[i] > conf_threshold:
                    bbox = result.boxes.xyxy[i]
                    
                    # Draw detection box
                    cv2.rectangle(frame, 
                                (int(bbox[0]), int(bbox[1])), 
                                (int(bbox[2]), int(bbox[3])), 
                                (255, 105, 180), 2)  # Pink for love
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center (handle NaN)
                    if (0 <= center_y < camera.height and 
                        0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            if distance < closest_distance:
                                human_detected = True
                                closest_distance = distance
                                closest_bbox = bbox
    
    # LOVE BEHAVIOR - follow human with differential drive
    if human_detected and closest_distance != float('inf'):
        # Calculate human position relative to frame center
        bbox_center_x = (closest_bbox[0] + closest_bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Calculate angle offset (normalized to -1 to 1)
        offset_x = bbox_center_x - frame_center_x
        max_offset = camera.width / 2
        normalized_angle = offset_x / max_offset  # Range: -1 (left) to 1 (right)
        
        # Convert to radians for differential drive (max Â±Ï€/4 for smooth turning)
        angle_rad = normalized_angle * (np.pi / 4)
        
        # Calculate differential drive speeds using cosine
        turning_factor = np.cos(angle_rad)
        
        # LOVE BEHAVIOR - approach and follow with differential drive
        if closest_distance < 450:
            # Too close - slight retreat
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'Too close {closest_distance:.0f}mm', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
            print(f'Love: Too close {closest_distance:.0f}mm - backing up', end='\r')
            
        elif closest_distance >= 450 and closest_distance <= 600:
            # Optimal distance - stop and enjoy proximity
            robot.stop()
            cv2.putText(depth_colormap, f'Perfect {closest_distance:.0f}mm <3', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2, cv2.LINE_AA)
            print(f'Love: Perfect distance {closest_distance:.0f}mm', end='\r')
            
        elif closest_distance > 600 and closest_distance <= 1200:
            # Medium distance - approach with differential steering
            base_speed = 0.3
            
            if angle_rad > 0.1:  # Human on right
                # Turn right: slow down right wheels
                left_speed = base_speed
                right_speed = base_speed * turning_factor
                robot.frontLeft(speed=left_speed)
                robot.backLeft(speed=left_speed)
                robot.frontRight(speed=right_speed)
                robot.backRight(speed=right_speed)
                cv2.putText(depth_colormap, f'Following right {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
                print(f'Love: Following right {closest_distance:.0f}mm (L:{left_speed:.2f} R:{right_speed:.2f})', end='\r')
                
            elif angle_rad < -0.1:  # Human on left
                # Turn left: slow down left wheels
                left_speed = base_speed * turning_factor
                right_speed = base_speed
                robot.frontLeft(speed=left_speed)
                robot.backLeft(speed=left_speed)
                robot.frontRight(speed=right_speed)
                robot.backRight(speed=right_speed)
                cv2.putText(depth_colormap, f'Following left {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
                print(f'Love: Following left {closest_distance:.0f}mm (L:{left_speed:.2f} R:{right_speed:.2f})', end='\r')
                
            else:  # Human centered
                # Move straight forward
                robot.forward(base_speed)
                cv2.putText(depth_colormap, f'Approaching {closest_distance:.0f}mm', 
                           (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
                print(f'Love: Approaching {closest_distance:.0f}mm (straight)', end='\r')
                
        else:  # closest_distance > 1200
            # Far away - approach faster with differential steering
            base_speed = 0.5
            
            if angle_rad > 0.1:  # Human on right
                # Turn right: slow down right wheels
                left_speed = base_speed
                right_speed = base_speed * turning_factor
                robot.frontLeft(speed=left_speed)
                robot.backLeft(speed=left_speed)
                robot.frontRight(speed=right_speed)
                robot.backRight(speed=right_speed)
                cv2.putText(depth_colormap, f'Searching right {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
                print(f'Love: Searching right {closest_distance:.0f}mm (L:{left_speed:.2f} R:{right_speed:.2f})', end='\r')
                
            elif angle_rad < -0.1:  # Human on left
                # Turn left: slow down left wheels
                left_speed = base_speed * turning_factor
                right_speed = base_speed
                robot.frontLeft(speed=left_speed)
                robot.backLeft(speed=left_speed)
                robot.frontRight(speed=right_speed)
                robot.backRight(speed=right_speed)
                cv2.putText(depth_colormap, f'Searching left {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
                print(f'Love: Searching left {closest_distance:.0f}mm (L:{left_speed:.2f} R:{right_speed:.2f})', end='\r')
                
            else:  # Human centered
                # Move straight forward quickly
                robot.forward(base_speed)
                cv2.putText(depth_colormap, f'Coming to you {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
                print(f'Love: Fast approach {closest_distance:.0f}mm (straight)', end='\r')
                
    else:
        # No human detected - stop and wait
        robot.stop()
        cv2.putText(depth_colormap, 'Waiting for human...', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2, cv2.LINE_AA)
        print('Love: No human detected - waiting', end='\r')
    
    # Display images
    scale = 0.1
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera
camera.observe(love_callback, names=['color_value'])

**To stop the camera and robot:**

In [ ]:
camera.stop()

**Behavior Description:**

The robot exhibits love behavior by following detected humans:

1. **Distance Measurement**: Calculates distance to closest detected human using depth image
2. **Intelligent Tracking**: 
   - Calculates human position relative to frame center
   - Only turns when human is >15% off-center (avoids continuous adjustment)
   - Turns left/right to recenter human in view
3. **Distance-Based Approach**:
   - **< 450mm**: Slight backward retreat (0.2 speed)
   - **450-600mm**: Stop - optimal "loving" distance
   - **600-1200mm**: Slow approach (0.3) or turn to track (0.3)
   - **> 1200mm**: Fast approach (0.5) or turn to track (0.4)
4. **Smart Following**: 
   - When centered: moves forward toward human
   - When off-center: rotates to recenter before/while approaching
   - When lost: stops and waits (no continuous searching)

The robot follows humans by maintaining visual tracking and optimal proximity, turning only when necessary to keep the human centered in its field of view.